<a href="https://colab.research.google.com/github/samibahig/recover/blob/master/Recover_CleaningData_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This script extract a data and metadata from the files
it gives dataframes and a labels file
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl

data_path = ''

#print('\nMETADATA :')
#metadata_filename = data_path + 'metadata.csv'
#meta_df = pd.read_csv(metadata_filename)
#meta_df.columns = ['#', 'plate', '-', 'symptoms'] + list(meta_df)[4:]
#print('available metadata :', list(meta_df))
#meta_idx = meta_df['ID'].to_list()
#meta_label = meta_df['symptoms'].to_list()
#meta_id_label_dict = {str(k): 1 if v=='S' else 0 for k, v in zip(meta_idx, meta_label)}

#DF1 : proteomics
print('\nPROTEOMICS DATA :')
proteomics_data_filename = data_path + '/content/proteomics.csv'

dim_df = pd.read_csv(proteomics_data_filename, nrows=1)
dim = len(list(dim_df))
print('# of columns in source csv file :', dim)
all_cols = [i for i in range(dim)]
feat_cols = all_cols[1:-4]
samplesidx_col = [0]

feat_df = pd.read_csv(proteomics_data_filename, skiprows=4, nrows=1, dtype=str, usecols=feat_cols)
features = list(feat_df)
print('# of features : ', len(features))
print('first feature :', features[0])
print('last feature :', features[-1])

idx_df = pd.read_csv(proteomics_data_filename, skiprows=6, index_col=0, skipfooter=4, usecols=[0], engine='python')
idx = list(idx_df.index.values)
print('# of idx : ', len(idx))
print('first id :', idx[0])
print('last id :', idx[-1])

df1 = pd.read_csv(proteomics_data_filename, skiprows=6, dtype=np.float32, skipfooter=4, usecols=feat_cols, engine='python')
assert df1.shape[0] == len(idx)
assert df1.shape[1] == len(features)

df1['idx'] = idx
df1.set_index('idx', inplace=True)
df1.columns = features
print('# of Nan values :', df1.isna().sum().sum())

#clean data of samples that are not in metadata :
#idx = df1.index.values
#y = []
#for k in range(len(idx)):
#    id = idx[k]
#    if id in meta_id_label_dict:
#        y.append(meta_id_label_dict[id])
#    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
#        y.append('to_remove')
#df1['label'] = y
#df1 = df1[df1.label != 'to_remove']

#create X and y matrices for ML :
y = list(df1['label'])
del df1['label']
X = df1.to_numpy()
print('proteomics data :')
print('# of samples : ', df1.shape[0])
print('# of features : ', df1.shape[1])
print('labels:', list(dict.fromkeys(y)))

# DF 2 : proteomics (cytokines)
print('\nPROTEOMICS CYTOKINES DATA :')
proteomics_cyt_data_filename = data_path + 'proteomics_cyt.csv'

dim_df = pd.read_csv(proteomics_cyt_data_filename, nrows=1)
dim = len(list(dim_df))
print('# of columns in source csv file :', dim)
all_cols = [i for i in range(dim)]
feat_cols = all_cols[1:-2]
samplesidx_col = [0]

feat_df = pd.read_csv(proteomics_cyt_data_filename, skiprows=4, nrows=1, dtype=str, usecols=feat_cols)
features = list(feat_df)
print('# of features : ', len(features))
print('first feature :', features[0])
print('last feature :', features[-1])

idx_df = pd.read_csv(proteomics_cyt_data_filename, skiprows=7, index_col=0, skipfooter=14, usecols=[0], engine='python')
idx = list(idx_df.index.values)
print('# of idx : ', len(idx))
print('first id :', idx[0])
print('last id :', idx[-1])

df2 = pd.read_csv(proteomics_cyt_data_filename, skiprows=7, dtype=np.float32, skipfooter=14, usecols=feat_cols, na_values=['> ULOQ'], engine='python')
assert df2.shape[0] == len(idx)
assert df2.shape[1] == len(features)

df2['idx'] = idx
df2.set_index('idx', inplace=True)
df2.columns = features
print('# of Nan values :', df2.isna().sum().sum())

#clean data of samples that are not in metadata :
idx = df2.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df2['label'] = y
df2 = df2[df2.label != 'to_remove']

#create X and y matrices for ML :
y = list(df2['label'])
del df2['label']
X = df2.to_numpy()
print('proteomics_cyt data :')
print('# of samples : ', df2.shape[0])
print('# of features : ', df2.shape[1])
print('labels:', list(dict.fromkeys(y)))

## an option to manage Nan values :
##replace nans with column mean :
##print('replacing nans with column mean')
#for col in list(df2):
#    #print(int(df[col].mean()))
#    df2[col].fillna(int(df2[col].mean()), inplace=True)
#
##print('is there NaN values ? :', df2.isnull().values.any())
##print('# of Nan values :', df2.isna().sum().sum())

## to concatenate the 2 proteomics dataframes if you want :
##df_1_2 = pd.concat([df1, df2], axis=1)

print('\nMETABOLOMICS DATA :')
metabolomics_data_filename = data_path + 'metabolomics.csv'
feat_df = pd.read_csv(metabolomics_data_filename, index_col=0, skiprows=[0], dtype=str, usecols=[0])
features = list(feat_df.index.values)
print('# of features : ', len(features))
print('first feature :', features[0])
print('last feature :', features[-1])

idx_df = pd.read_csv(metabolomics_data_filename, header=1, nrows=1)
idx = list(idx_df)[1:]
idx = [l[17:22] for l in idx]
print('# of idx : ', len(idx))

labels_df = pd.read_csv(metabolomics_data_filename, nrows=1)
labels = list(labels_df)[1:]
print('# of labels : ', len(labels))

cols_df = pd.read_csv(metabolomics_data_filename, header=1, nrows=1)
cols_list = list(cols_df)

df3 = pd.read_csv(metabolomics_data_filename, header=1, dtype=np.float32, na_values=['#DIV/0!'], usecols=cols_list[1:])
df3 = df3.T
df3['idx'] = idx
df3.set_index('idx', inplace=True)
df3.columns = features
df3 = df3.dropna(axis=1)

#clean data of samples that are not in metadata :
idx = df3.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df3['label'] = y
df3 = df3[df3.label != 'to_remove']

#create X and y matrices for ML :
y = list(df3['label'])
del df3['label']
X = df3.to_numpy()

print('metabolomics data :')
print('# of samples : ', df3.shape[0])
print('# of features : ', df3.shape[1])
print('labels:', list(dict.fromkeys(y)))

## to concatenate the 2 proteomics dataframes and the metabolomics if you want :
##df = pd.concat([df_1_2, df3], axis=1)
##df = df.dropna(axis=0)
##print('multi-omics df :')
##print('# of samples : ', df.shape[0])
##print('# of features : ', df.shape[1])

## save X and y in pickles if you want :
##data_name = 'recover_multiomics_'
##feat_dict = {k: str(v) for k, v in zip(range(len(list(df))), list(df))}
##with open(data_path + data_name + 'feat_dict', 'wb') as fo:
##    pkl.dump(feat_dict, fo)
##with open(data_path + data_name + 'X', 'wb') as fo:
##            pkl.dump(X, fo)
##with open(data_path + data_name + 'y', 'wb') as fo:
##            pkl.dump(y, fo)


PROTEOMICS DATA :


FileNotFoundError: ignored

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl

dataframe = pd.read_csv('/metabolomics.csv', index_col=0)
desc = dataframe.describe()
desc

,QC,QC.1,QC.2,QC.3,QC.4,QC.5,QC.6,QC.7,QC.8,QC.9,QC.10,QC.11,QC.12,QC.13,QC.14,QC.15,QC.16,QC.17,QC.18,QC.19,QC.20,QC.21,QC.22,QC.23,QC.24,QC.25,F_persistent,F_persistent.1,F_persistent.2,F_persistent.3,F_persistent.4,F_persistent.5,F_persistent.6,F_persistent.7,F_persistent.8,F_persistent.9,F_persistent.10,F_persistent.11,F_persistent.12,F_persistent.13,...,M_persistent.9,M_persistent.10,M_NoSymptoms.7,M_persistent.11,M_persistent.12,M_NoSymptoms.8,M_NoSymptoms.9,M_persistent.13,M_persistent.14,M_persistent.15,M_NoSymptoms.10,M_NoSymptoms.11,M_persistent.16,Unnamed: 156,Unnamed: 157,M_persistent.17,M_persistent.18,Unnamed: 160,M_persistent.19,M_persistent.20,M_persistent.21,M_persistent.22,M_NoSymptoms.12,M_NoSymptoms.13,M_NoSymptoms.14,M_persistent.23,M_persistent.24,M_persistent.25,M_NoSymptoms.15,M_NoSymptoms.16,M_persistent.26,M_NoSymptoms.17,M_NoSymptoms.18,M_NoSymptoms.19,M_persistent.27,M_NoSymptoms.20,M_NoSymptoms.21,M_persistent.28,M_NoSymptoms.22,M_persistent.29
count,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,...,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069,4069
unique,3825,3866,3843,3873,3889,3878,3907,3879,3887,3893,3889,3894,3910,3904,3921,3913,3906,3911,3927,3945,3934,3932,3933,3934,3958,3927,3866,3869,3826,3880,3838,3837,3831,3840,3856,3849,3814,3775,3834,3792,...,3818,3885,3804,3785,3870,3883,3851,3841,3844,3853,3862,3807,3863,3855,3854,3855,3692,3836,3817,3838,3873,3866,3891,3816,3832,3887,3870,3872,3817,3876,3831,3830,3890,3873,3884,3818,3894,3826,3813,3837
top,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
freq,188,147,170,140,124,135,106,136,127,119,123,119,107,115,101,110,117,109,93,76,90,92,91,90,66,97,147,144,187,133,175,176,182,173,157,164,199,238,179,221,...,195,128,209,228,143,130,162,172,169,160,151,206,150,158,159,158,321,177,196,175,140,147,122,197,181,126,143,141,196,137,182,183,123,140,129,195,119,187,200,176


In [ ]:
# print(dataframe)
~dataframe.isin(['#DIV/0!', np.inf, -np.inf]).any(1)
#dataframe = dataframe[~dataframe.isin(['#DIV/0!', np.inf, -np.inf]).any(1)]

Group
Name                 True
3.15_160.1695m/z     True
8.56_366.6297m/z     True
6.52_828.4067n       True
7.23_281.0516m/z     True
                     ... 
0.49_1030.2676m/z    True
0.49_765.5065n       True
9.19_306.2746m/z     True
0.49_1047.2211n      True
7.71_483.0063m/z     True
Length: 3990, dtype: bool

In [ ]:
dataarray = dataframe.to_numpy()[1:] 
dataarray

array([['3.694739295768', '3.35860528161234', '4.12748808249481', ...,
        '0', '0', '0'],
       ['0', '0', '1.26864417277367', ..., '0', '0', '0'],
       ['19.0686397478299', '18.9035458274785', '17.7168613273485', ...,
        '0', '0', '0'],
       ...,
       ['62.9500617087009', '38.7682784708816', '27.4077738901859', ...,
        '23.7604303172762', '42.7249274457891', '35.2522300098789'],
       ['881.75642011388', '995.161025158501', '933.71637425069', ...,
        '1232.79159020343', '1247.02474805189', '1212.89384027735'],
       ['6.34260184664678', '5.75329845418387', '6.30466534125239', ...,
        '0', '1.15204230435919', '2.88273732258928']], dtype=object)

In [ ]:
float_array = dataarray.astype(np.float)

In [ ]:
float_array

array([[3.69473930e+00, 3.35860528e+00, 4.12748808e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.26864417e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.90686397e+01, 1.89035458e+01, 1.77168613e+01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [6.29500617e+01, 3.87682785e+01, 2.74077739e+01, ...,
        2.37604303e+01, 4.27249274e+01, 3.52522300e+01],
       [8.81756420e+02, 9.95161025e+02, 9.33716374e+02, ...,
        1.23279159e+03, 1.24702475e+03, 1.21289384e+03],
       [6.34260185e+00, 5.75329845e+00, 6.30466534e+00, ...,
        0.00000000e+00, 1.15204230e+00, 2.88273732e+00]])

In [ ]:
list_ = [False, True, True, True]
any(list_)

True

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split